# Example for running a multislice simulation with TEMSIM
This is a comprehensive example on how to use the python wrapper class `Multislice` to run a multislice simulation with the TEMSIM package.
- [Basic setup](#Setting-up-a-simulation)
    - [Running a simulation](#Running-the-simulation)
    - [Monitoring the simulation](#Monitoring-the-simulation)
    - [Diffraction pattern](#Diffraction-pattern-and)
    - [Beam evolution](#Beam-evolution-as-a-function-of-thickness)
    - [Loading a simulation](#Loading-a-simulation)
    - [Thickness dependent diffraction patterns](#Thickness-dependent-diffraction-patterns)

In [ ]:
import multislice.multislice as mupy       # the temsim wrapper module
from utils import*                         # utilities
path = 'data/test/'                        # where the data are held for this example

## Setting up a simulation 
For this example,  Electron Diffraction of Silicon in the zone axis [110] orientation is simulated at $200keV$. The program **autoslic** of the TEMSIM package is going to be used which assumes that the sample is not periodic along the beam axis.

### Choosing the simulation type
- `path` : This is the working directory for the simulation. The basename of the folder is used as a prefix for the name of the simulation. The naming convention for the produced files is : *folder_name*\_*tag*\_*program*.
- `tag` : An optional string to be used in the naming convention. We will used 'example_0' as the tag.
- `mulslice` : We consider that the sample is not periodic along the beam. The program **autoslic** will be used hence setting `mulslice=False`.
- `data` : The file containing the crystal information, in particular the atomic coordinates oriented in such a way that the beam is along the $z$ direction. 


In [ ]:
args = {'name':path,'mulslice':False,'data':'Si110.xyz','tag':'example_0'}

### Specifying the parameters  
- `keV` : The simulation is performed at $200keV$
- `repeat` : The super cell setup in x,y and z. Here the crystal is a 2x2 in the transverse plane and 10 unit cells thick.
- `NxNy` : The real space sampling for the electrostatic potential. This also happens to be the reciprocal space sampling (or number of beams) as the multislice algorithm uses the discrete Fourier transform (DFT). 
- `slice_thick` : the thickness of the slices.
- `Nhk` : The beams index which are going to be recorded as the incident beam propagates along the sample. `Nhk=3` means that the beams $h,k=(0,0),(1,0),(0,1),(1,1)...$ up to $h,k=(3,3)$ are going to be recorded. Note that using this convention, since the super cell is $2\times 2$ the actual beam recorded for index $h,k=(1,1)$ is $q_x,q_y=(2,2)$.
- `i_slice` : the diffraction patterns will be save ever **i_slice** slices. 

In [ ]:
args.update({'keV':200,'repeat':[2,2,100],'NxNy':512,'slice_thick':1.91,'Nhk':6,'i_slice':20})

### Running the simulation 
The simulation can be run by creating an instance of the `Multislice` wrapper class. 

The argument `opt='srf'` is used to : 
- 's' : save the instanciated Multislice object into a .pkl  
- 'r' : create a .sh job, a .in input file fed to TEMSIM  and run the job 
- 'f' : force the simulation to be run again even though it has already been done in the past. 

The argument `ssh` provides the name of the machine where the job is to be run. Here we specify `badb` which is registered as a cluster and will therefore run the job through the sun grid engine.

In [ ]:
multi=mupy.Multislice(opt='sr',ssh='badb',**args)

You can see the content of the generated files by calling : 
- `print_datafiles` : for the .xyz file
- `print_decks` : the input file fed to temsim 
- `print_job` : the job that was submitted 


In [ ]:
multi.print_datafiles()
multi.print_decks()
multi.print_job()

### Monitoring the simulation 
if you want to check the status of the simulation, `wait_simu()` can be called.

In [ ]:
multi.wait_simu()

TEMISIM will generate a **.log** file with progress on the simulation. The content of this output can be viewed by calling `print_log()`. For each slice, the number of coordinates can be seen as well as the integrated intensity. Since the value is close to 1, this means that the sampling is good enough. 

In [ ]:
multi.print_log() 

### Diffraction pattern
The diffraction pattern can be seen by calling `pattern`. 

In [ ]:
multi.pattern(Iopt='',caxis=[0,30000])

The argument `Iopt` gives some additional options for better rendering : 
- n(normalize): normalize the intensity so that the intensity of the brightest peak is 1.  
- s(fftshift) : use fftshift to display 
- c(crop)     : crop up to `Nmax` pixel. If `tol` is set, then crop up to $I<tol$.
- l(log)      : display colormap in logscale

In [ ]:
fig,((ax1,ax2),(ax3,ax4)) = dsp.create_fig(rc='22')
cmap,fonts = 'binary',{'lab':12,'title':12,'tick':10}
multi.pattern(Iopt=''             ,caxis=[0,30000],pOpt='t',cmap=cmap,fonts=fonts,title='raw',ax=ax1,fig=fig)
multi.pattern(Iopt='sn'           ,caxis=[0,0.01] ,pOpt='t',cmap=cmap,fonts=fonts,title='normalize,shift',ax=ax2,fig=fig)
multi.pattern(Iopt='snc',Nmax=50  ,caxis=[0,0.01] ,pOpt='t',cmap=cmap,fonts=fonts,title='crop to Nmax=100',ax=ax3,fig=fig)
multi.pattern(Iopt='cnsl',tol=1e-4,caxis=[-5,0]   ,pOpt='t',cmap=cmap,fonts=fonts,title='logscale and crop to tol',ax=ax4,fig=fig)

### Beam evolution as a function of thickness 
The evolution of the main important beam as a function of propagation into the sample is display by calling `beam_vs_thickness`. 
To include the central beam, `orig` must be set. 

The generic option 's' can be used to save the figure in the `name`. 


In [ ]:
multi.beam_vs_thickness(orig=True,opt='ps',name='figures/example0_beams.svg',fonts={'tick':20})

## Loading a simulation 
A Multislice object previously saved can be loaded with the `load` function from the `postprocess` library. Just indicate the path of the simulation and the `tag` of the simulation if one was specified. Here, we used it to simply check what version of the Multislice class was used to produce the object.

In [ ]:
import multislice.postprocess as pp        # postprocessing library
multi = pp.load(path,'example_0',v=2)      # verbose option calls log_info()
print('Multislice version:',multi.version)

## Thickness dependent diffraction patterns

The individual diffraction patterns are displayed by specfiyng the thickness id `iz`.
Here we also use the generic option `opt=sc` to save and close the figure. The figures are saved in `name`.

In [ ]:
pargs={'Iopt':'sN','pOpt':'Xt','caxis':[0,0.2],'xylims':1,'cmap':'viridis'}
for iz in [27,65]:
    multi.pattern(iz=iz,name='figures/example0_pattern%s.png' %str(iz).zfill(4),opt='sc',fonts={'title':35},**pargs)

beams | p1  | p2  | 
----- | --- | --- | 
![](figures/example0_beams.svg) | ![](figures/example0_pattern0027.png) | ![](figures/example0_pattern0065.png)

Note how at $106$ the intensities of the main reflections have significantly increased at the expense of the central beam while at $252A$ most of the overall brightness has been scattered back into the central beam. This is an obvious manifestation of dynamical diffraction. 

It is also possible to save the thickness dependent patterns in a **.gif** with `patterns2gif`.


In [ ]:
multi.patterns2gif('figures/example0_patterns.gif',v=1,**pargs)

beams | gif pattern
----- | -----------
![](figures/example0_beams.svg) | ![](figures/example0_patterns.gif)


## Resuming a simulation 
It is possible to resume a simulaton from a previously run state.


In [ ]:
#multi.resume(Nz=100)